<a href="https://colab.research.google.com/github/Salmaag/Learn-Arabic/blob/main/Demo/Beam_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from math import tanh
from numpy import array
from numpy import argmax
from tqdm import tqdm_notebook

In [2]:
def sigmoid(x):
  return 1/(1 + np.exp(-x)) 

In [3]:
def test(model, test_loader, max_text_length):
  k= 3
  model.eval()
  predictslmao = []
  gt = []
  imgs = []

  class Breakit(Exception): pass

  with torch.no_grad():
    for batch in  tqdm_notebook(test_loader):
      output_sequences = [([], 0.0)]
      src, trg = batch
      imgs.append(src.flatten(0,1))
      src, trg = src.cuda(), trg.cuda()            
      memory = get_memory(model,src.float())
      out_indexes = [None] * k
      allScores = [None] * k
      for l in range(k):
        out_indexes[l] = [tokenizer.chars.index('SOS'), ]
        allScores[l] = [0.0]
      mask = model.generate_square_subsequent_mask(1).to('cuda')
      trg_tensor = torch.LongTensor(out_indexes[0]).unsqueeze(1).to(device)
      outputfirst = model.vocab(model.transformer.decoder(model.query_pos(model.decoder(trg_tensor)), memory,tgt_mask=mask))
      mean = torch.mean(outputfirst[0][0])
      std = torch.std(outputfirst[0][0])            
      outputfirst = torch.tensor([((x.item()) - mean.item()) / (std.item()) for x in outputfirst[0][0]])
      try:
        for i in range(100):
          list2 = []
          if i != 0:
            outputs = []
            mask = model.generate_square_subsequent_mask(i + 1).to('cuda')
            for iter in range(k):
              trg_tensor = torch.LongTensor(out_indexes[iter]).unsqueeze(1).to(device)
              output = model.vocab(model.transformer.decoder(model.query_pos(model.decoder(trg_tensor)), memory,tgt_mask=mask))
              mean = torch.mean(output[i][0])
              std = torch.std(output[i][0])            
              output = torch.tensor([((x.item()) - mean.item()) / (std.item()) for x in output[i][0]])
              outputs.append(output)
          for m in range(100):
            if i != 0: 
              for iter in range(k):
                old_seq, old_score = output_sequences[iter]
                new =  [m]
                new_sc = old_score + sigmoid(outputs[iter][m].item())
                list2.append((new, new_sc))
            else:
              new = [] + [m]
              new_sc = 0.0 +sigmoid(outputfirst[m].item())
              list2.append((new, new_sc))
          output_sequences = sorted(list2, key = lambda val: val[1], reverse = True)
          output_sequences = output_sequences[:k]          
          for n in range(k):
            out_indexes[n].append(output_sequences[n][0][0])
            allScores[n][-1] =  output_sequences[n][1]     
            if output_sequences[n][0][0] == tokenizer.chars.index('EOS'):
              if k == 1:
                raise Breakit
              else:
                output_sequences[n], output_sequences[n-1] =  output_sequences[n-1], output_sequences[n]
                k -= 1 

      except Breakit:
        pass

      predicts = [[list(), 0.0],[list(), 0.0],[list(), 0.0]] 
      for b in range(k):
        predicts[b][0].append(tokenizer.decode(out_indexes[b]))
        predicts[b][1] = allScores[b]
      predictslmao.append(predicts)
      gt.append(tokenizer.decode(trg.flatten(0,1)))
  return predictslmao, gt, imgs

In [ ]:
test_loader = torch.utils.data.DataLoader(DataGenerator(source_path,charset_base,max_text_length,'test',transform), batch_size=1, shuffle=False, num_workers=2)
predicts, gt, imgs = test(model, test_loader, max_text_length)